<a href="https://colab.research.google.com/github/Nerothemadlad/AI-basics/blob/main/Identifiaction%20and%20Decision%20Tree/Identification_Tree_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files, data_table
import pandas as pd
import io

data_table.enable_dataframe_formatter()
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(io.BytesIO(uploaded[filename]))
df

Saving Book3.xlsx to Book3.xlsx


,ID,Thời gian,Loại giải,Mặt sân,Dùng hết sức,Kết quả
0,1,Sáng,Master,Sân cỏ,True,Thắng
1,2,Chiều,Grand slam,Đất nện,True,Thắng
2,3,Tối,Giao hữu,Sân cứng,False,Thắng
3,4,Chiều,Giao hữu,Mixed,False,Thua
4,5,Chiều,Master,Đất nện,True,Thua
5,6,Chiều,Grand slam,Sân cỏ,True,Thắng
6,7,Chiều,Grand slam,Sân cứng,True,Thắng
7,8,Chiều,Grand slam,Sân cứng,True,Thắng
8,9,Sáng,Master,Sân cỏ,True,Thắng
9,10,Chiều,Grand slam,Đất nện,True,Thua


In [ ]:
import numpy as np

def entropy_by_category(df: pd.DataFrame, category: str, class_column: str) -> float:
    """
    Parameters:
    -----------
    df: a pandas DataFrame
    category: The column by whose unique values the input dataset will be categorized
    class_column: The column with the classification labels

    Returns:
    --------
    total_entropy: Total weighted entropy (magnitude of disorder) of categorized subsets
    """
    total_entropy = 0
    # print(f"Category: {category}")

    # Loop over all possible categorizing values
    for val in df[category].unique():
      # Filter all entries based on a categorizing value
      filtered_df = df[df[category] == val]
      tallies =  filtered_df[class_column].value_counts(normalize=False)
      # print(f"{val}:")
      # print(tallies)|
      # Count entries by labels and normalize to get probabilites
      probs = filtered_df[class_column].value_counts(normalize=True).to_numpy()
      # Calculate entropy of the subset branch and weight it by its sample size
      entropy_by_val = np.sum(probs * np.log2(1/probs))
      # print(f"Entropy: {entropy_by_val}")
      weight = len(filtered_df)/len(df)
      weighted_entropy_by_val = weight * entropy_by_val
      # Add the weighted entropy to total entropy of category branch
      total_entropy += weighted_entropy_by_val



    return total_entropy

In information theory, $E[I] =-\sum_{i}^{n}p_ilog_{2}(p_i)$ is the expected value of information. $1/p_i$ measures by how much an event space $i$ divides the sample space, i.e. informativeness of $i$. $log_{n}(1/p_i)$ measures how many units of information $i$ would provide (each unit of information divides the sample space by $n$).

The $entropy$ get smaller if some events become more dominant/likely than the rest, i.e. less randomness in the distribution. The base $2$ is chosen mainly because the binary nature of the computer makes it easier to compute/estimate (as easy as counting bits).

In [ ]:
def discrete_argmin(mapping: dict[str, float]) -> str:
    """
    Parameters:
    -----------
    mapping: a dictionary containing the sets of input and corresponding output

    Returns:
    --------
    min_arg: The input whoses output is minimal
    """
    min_arg = min(mapping, key=mapping.get)

    return min_arg

In [ ]:
# Input name of the column with classifying label here
CLASS_COLUMN = 'Kết quả'
CATEGORIES = tuple(df.loc[:, df.columns!=CLASS_COLUMN].columns[1:])

def test_category_selection(df: pd.DataFrame, categories: list[str]) -> str:
    """
    Parameters:
    -----------
    mapping: a dictionary containing the sets of input and corresponding output

    Returns:
    --------
    test: The category for used in next test
    """
    entropy_dict = {}

    for category in categories:
        entropy_dict[category] = entropy_by_category(df, category, CLASS_COLUMN)

    test = discrete_argmin(entropy_dict)
    return test, entropy_dict

In [ ]:
TREE_LEVELS = 3
categories_for_level = list(CATEGORIES)

for level in range(TREE_LEVELS):
    level_test, level_entropy_dict = test_category_selection(df, categories_for_level)
    print(f"Level {level+1} test: {level_test}")
    print(f"Level {level+1} entropy: {level_entropy_dict}")
    categories_for_level.remove(level_test)


Level 1 test: Mặt sân
Level 1 entropy: {'Thời gian': 0.8223219282981569, 'Loại giải': 0.806037309514985, 'Mặt sân': 0.30342206076708395, 'Dùng hết sức': 0.8957049265635535}
Level 2 test: Loại giải
Level 2 entropy: {'Thời gian': 0.8223219282981569, 'Loại giải': 0.806037309514985, 'Dùng hết sức': 0.8957049265635535}
Level 3 test: Thời gian
Level 3 entropy: {'Thời gian': 0.8223219282981569, 'Dùng hết sức': 0.8957049265635535}


ML and its subfields have borrowed terms and equations of other fields like graph theory (in seacrh algorithm), probability theory (in Bayesian inference), information theory (in identification/decision tree), multivariable calculus and especially linear algebra (in deep neural network). And yet somehow, the reasons behind the borrowing of these equations are oftentimes poorly explained, which makes them seem abstractly chosen.

For example, the embedding layer in LLMs (representing its vocabs) is a matrix of high dimensional vectors denoting words/morphemes/punctuations, where most dimensions are associated with some semantic features. Although posing a risk of redundancy, the more dimensions the better, e.g. $cat = a[animate] + b[feline] + c[domesticated] + d[small]$ is better than $cat = a[animate] + b[feline]$. LLMs may soon reach its limit when computing power growth stagnates and information contamination growths.